In [2]:
import pandas as pd
import numpy as np

In [4]:
from gliner import GLiNER

In [2]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
# model = AutoModelForTokenClassification.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
# ner = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
# example = "서울역으로 안내해줘. 삼성전자에서는 NPU를 사용한 새로운 스마트 워치를 출시하였다"
# ner_results = ner(example)
# print(ner_results)

In [3]:
df_naver = pd.read_csv('../data/naver_news.csv')
df_daum = pd.read_csv('../data/daum_news.csv')

In [5]:
model = GLiNER.from_pretrained("taeminlee/gliner_ko")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\s_torileeo99\AppData\Local\anaconda3\envs\trans\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\s_torileeo99\AppData\Local\anaconda3\envs\trans\lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\Users\s_torileeo99\AppData\Local\anacon

In [ ]:
text = """
피터 잭슨 경(, 1961년 10월 31일 ~ )은 뉴질랜드의 영화 감독, 각본가, 영화 프로듀서이다. J. R. R. 톨킨의 소설을 원작으로 한 《반지의 제왕 영화 3부작》(2001년~2003년)의 감독으로 가장 유명하다. 2005년에는 1933년작 킹콩의 리메이크작 《킹콩(2005)》의 감독을 맡았다.
"""

tta_labels = ["ARTIFACTS", "ANIMAL", "CIVILIZATION", "EVENT", "STUDY_FIELD", "LOCATION", "MATERIAL", "ORGANIZATION", "PERSON", "PLANT", "TERM", "THEORY"]

entities = model.predict_entities(text, tta_labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [33]:
news_labels = ["ARTIFACTS", "ANIMAL", "CIVILIZATION", "EVENT", "STUDY_FIELD", "LOCATION", "MATERIAL", "ORGANIZATION", "PERSON", "PLANT", "TERM", "THEORY"]

In [7]:
df_naver.dropna(subset=['title', 'content', 'publication_date'], inplace=True)
df_daum.dropna(subset=['title', 'content', 'publication_date'], inplace=True)

In [18]:
def convert_date_naver(date):
    date = date.split('. ')[0]
    if date == 'Null':
        return np.nan
    return pd.to_datetime(date, format='%Y.%m.%d').date()

In [ ]:
df_naver['publication_date'] = df_naver['publication_date'].apply(convert_date_naver)

In [19]:
def convert_date_daum(date):
    date = date.split('. ')[:-1]
    date = '.'.join(date)
    if date == 'Null':
        return np.nan
    return pd.to_datetime(date, format='%Y.%m.%d').date()

In [21]:
df_daum['publication_date'] = df_daum['publication_date'].apply(convert_date_daum)

In [24]:
df_naver.dropna(subset=['title', 'content', 'publication_date'], inplace=True)
df_daum.dropna(subset=['title', 'content', 'publication_date'], inplace=True)

In [26]:
start_date = pd.to_datetime('2024-06-01').date()
end_date = pd.to_datetime('2024-09-01').date()
filtered_naver = df_naver[(df_naver['publication_date'] >= start_date) & (df_naver['publication_date'] <= end_date)]
filtered_daum = df_daum[(df_daum['publication_date'] >= start_date) & (df_daum['publication_date'] <= end_date)]

In [28]:
filtered_naver.info()
filtered_daum.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21972 entries, 65156 to 87862
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   url               21972 non-null  object
 1   title             21972 non-null  object
 2   author            21972 non-null  object
 3   publication_date  21972 non-null  object
 4   update_date       21972 non-null  object
 5   publisher_id      21972 non-null  object
 6   category_id       21972 non-null  object
 7   platform_id       21972 non-null  object
 8   content           21972 non-null  object
dtypes: object(9)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 64998 entries, 190806 to 264730
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   article_url       64998 non-null  object
 1   title             64998 non-null  object
 2   author            64998 non-null  object
 3   pub

In [29]:
batch_size = 100  # Adjust batch size based on your task
batches_naver = [filtered_naver['title'][i:i + batch_size] for i in range(0, len(filtered_naver['title']), batch_size)]
batches_daum = [filtered_daum['title'][i:i + batch_size] for i in range(0, len(filtered_daum['title']), batch_size)]

In [30]:
def predict_entity(title, news_labels):
    entities = model.predict_entities(title, news_labels)
    return [entity['text'] for entity in entities]

In [31]:
def process_batch(batch):
    return batch.apply(lambda title: predict_entity(title, news_labels))

In [34]:
entities_list_naver = []
i = 1
for batch in batches_naver:
    print(f'--------batch {i} in process--------')
    entities_list_naver.extend(process_batch(batch))
    i += 1

filtered_naver['entities'] = entities_list_naver

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


--------batch 1 in process--------
--------batch 2 in process--------
--------batch 3 in process--------
--------batch 4 in process--------
--------batch 5 in process--------
--------batch 6 in process--------
--------batch 7 in process--------
--------batch 8 in process--------
--------batch 9 in process--------
--------batch 10 in process--------
--------batch 11 in process--------
--------batch 12 in process--------
--------batch 13 in process--------
--------batch 14 in process--------
--------batch 15 in process--------
--------batch 16 in process--------
--------batch 17 in process--------
--------batch 18 in process--------
--------batch 19 in process--------
--------batch 20 in process--------
--------batch 21 in process--------
--------batch 22 in process--------
--------batch 23 in process--------
--------batch 24 in process--------
--------batch 25 in process--------
--------batch 26 in process--------
--------batch 27 in process--------
--------batch 28 in process--------
-

C:\Users\s_torileeo99\AppData\Local\Temp\ipykernel_17148\2503790258.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_naver['entities'] = entities_list_naver


In [36]:
entities_list_daum = []
j = 1
for batch in batches_daum:
    print(f'--------batch {j} in process--------')
    entities_list_daum.extend(process_batch(batch))
    j += 1

filtered_daum['entities'] = entities_list_daum

--------batch 1 in process--------
--------batch 2 in process--------
--------batch 3 in process--------
--------batch 4 in process--------
--------batch 5 in process--------
--------batch 6 in process--------
--------batch 7 in process--------
--------batch 8 in process--------
--------batch 9 in process--------
--------batch 10 in process--------
--------batch 11 in process--------
--------batch 12 in process--------
--------batch 13 in process--------
--------batch 14 in process--------
--------batch 15 in process--------
--------batch 16 in process--------
--------batch 17 in process--------
--------batch 18 in process--------
--------batch 19 in process--------
--------batch 20 in process--------
--------batch 21 in process--------
--------batch 22 in process--------
--------batch 23 in process--------
--------batch 24 in process--------
--------batch 25 in process--------
--------batch 26 in process--------
--------batch 27 in process--------
--------batch 28 in process--------
-

C:\Users\s_torileeo99\AppData\Local\Temp\ipykernel_17148\737660432.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_daum['entities'] = entities_list_daum


In [38]:
filtered_daum.tail()

,article_url,title,author,publication_date,content,publisher_id,category_id,platform_id,entities
264726,https://v.daum.net/v/20240829005018549,"‘일본풍 미소녀’ 서브컬처 게임, 국내시장 휩쓴다",안상현 기자,2024-08-29,"이 글자크기로 변경됩니다. (예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소...",조선일보,IT,다음,"[일본풍, 미소녀, 서브컬처 게임]"
264727,https://v.daum.net/v/20240829005014548,“AI가 사진 2만개·데이터 40만개 학습… 치과 의사 탐색률 50% 높여”,유지한 기자,2024-08-29,"이 글자크기로 변경됩니다. (예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소...",조선일보,IT,다음,"[AI가, 치과, 의사]"
264728,https://v.daum.net/v/20240829003538450,"아마존, 전면 개편한 ‘알렉사’ 10월 출시",장형태 기자,2024-08-29,"이 글자크기로 변경됩니다. (예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소...",조선일보,IT,다음,"[아마존, 알렉사]"
264729,https://v.daum.net/v/20240829000311104,"2D 도트 게임의 귀환, 'THE 라그나로크' 9월 9일 국내 출시",김진두,2024-08-29,초기 2D 온라인 게임으로 큰 인기를 끌었던 '라그나로크'가 모바일 게임으로 재탄생...,YTN,IT,다음,"[2D, 도트 게임의, THE 라그나로크]"
264730,https://v.daum.net/v/20240829004049484,AI가 더 빠르고 정확해진다… 차세대 반도체 쏟아져,이해인 기자,2024-08-29,"이 글자크기로 변경됩니다. (예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소...",조선일보,IT,다음,"[AI가, 반도체]"


In [39]:
filtered_naver.to_csv('../data/filtered_naver.csv', index=False)
filtered_daum.to_csv('../data/filtered_daum.csv', index=False)